# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    #1. cleaning nhl data
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nhl_df_2018 = nhl_df[nhl_df['year']==2018]
    remove_index = [0,9,18,26] #to remove rows with characters
    nhl_df_2018 = nhl_df_2018[~nhl_df_2018.index.isin(remove_index)]
    nhl_df_2018["ratio"] = pd.to_numeric(nhl_df_2018["W"]) / ( pd.to_numeric(nhl_df_2018["W"])+ pd.to_numeric(nhl_df_2018["L"]))
    nhl_df_2018_ratio = nhl_df_2018[['team','ratio']]
    nhl_df_2018_ratio.replace('\*','', regex=True, inplace=True)
    
    #2. cities data
    cities.rename(columns={'Metropolitan area':'area' ,'Population (2016 est.)[8]':'pop'}, inplace=True)
    cities_nhl = cities[['area','pop','NHL']]
    cities_nhl.replace('\[note \d*\]', '', regex=True, inplace=True)
    cities_nhl.replace('', np.NaN, inplace=True)
    cities_nhl.replace('—', np.NaN, inplace=True)
    cities_nhl = cities_nhl[~cities_nhl['NHL'].isna()]
    
    #3. changing area name in nhl data
    one_name_area = ['Detroit','Columbus','Vegas','Toronto']
    nhl_df_2018_ratio['area'] =  nhl_df_2018_ratio['team'].apply(lambda x : re.split(' ',x)[0] if (len(re.split(' ',x)) == 2 or re.split(' ',x)[0] in one_name_area) else ' '.join(re.split(' ',x)[:2]))
    change_val = {'San Jose' : 'San Francisco Bay Area'
             , 'Anaheim' : 'Los Angeles'
             , 'New Jersey' : 'New York City'
             , 'Minnesota' : 'Minneapolis–Saint Paul'
             , 'Florida' : 'Miami–Fort Lauderdale'
             , 'Arizona' : 'Phoenix'
             , 'Carolina' : 'Raleigh'
             , 'Colorado' : 'Denver'
             , 'New York' : 'New York City' #metropolitan 기준으로 수정
             , 'Dallas' : 'Dallas–Fort Worth'
             , 'Washington' : 'Washington, D.C.'
             , 'Vegas' : 'Las Vegas'
             , 'Tampa Bay' : 'Tampa Bay Area' }
    nhl_df_2018_ratio.replace({'area' : change_val }, inplace=True)
    
    cities_nhl.sort_values(by='area')

    #5. group by and merge
    nhl_area_grp =  nhl_df_2018_ratio.groupby("area", as_index=False).agg({"ratio" : np.average})
    nhl_df = pd.merge(cities_nhl, nhl_area_grp, how = 'left', on = 'area')
    nhl_df = nhl_df.astype({'pop': int, 'ratio': float})
    
#     raise NotImplementedError()
    
    population_by_region = list(nhl_df['pop']) # pass in metropolitan area population from cities
    win_loss_by_region =  list(nhl_df['ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [10]:
nhl_correlation()

0.012486162921209912

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    #1. cleaning nba data
    nba_df_2018 = nba_df[nba_df['year']==2018]
    nba_df_2018["ratio"] = pd.to_numeric(nba_df_2018["W"]) / ( pd.to_numeric(nba_df_2018["W"])+ pd.to_numeric(nba_df_2018["L"]))
    nba_df_2018 = nba_df_2018[['team','ratio']]
    nba_df_2018.replace('\*','', regex=True, inplace=True)
    nba_df_2018.replace('\s\(\d+\)','', regex=True, inplace=True)
    nba_df_2018_ratio = nba_df_2018[['team','ratio']]
    
    #2. cities data
    cities.rename(columns={'Metropolitan area':'area' ,'Population (2016 est.)[8]':'pop'}, inplace=True)
    cities_nba = cities[['area','pop','NBA']]
    cities_nba.replace('\[note \d*\]', '', regex=True, inplace=True)
    cities_nba.replace('', np.NaN, inplace=True)
    cities_nba.replace('—', np.NaN, inplace=True)
    cities_nba = cities_nba[~cities_nba['NBA'].isna()]
    cities_nba.sort_values(by='area', inplace = True)
    
    #3. changing area name in nhl data
    one_name_area = ['Portland'] 
    nba_df_2018_ratio['area'] =  nba_df_2018_ratio['team'].apply(lambda x : re.split(' ',x)[0] if (len(re.split(' ',x)) == 2 or re.split(' ',x)[0] in one_name_area) else ' '.join(re.split(' ',x)[:2]))
    change_val = {'Brooklyn' : 'New York City'
             , 'Dallas' : 'Dallas–Fort Worth'
             , 'Golden State' : 'San Francisco Bay Area'
             , 'Indiana' : 'Indianapolis'
             , 'Miami' : 'Miami–Fort Lauderdale'
             , 'Minnesota' : 'Minneapolis–Saint Paul'
             , 'New York' : 'New York City'
             , 'Utah' : 'Salt Lake City'
             , 'Washington' : 'Washington, D.C.'}
    nba_df_2018_ratio.replace({'area' : change_val }, inplace=True)
    nba_df_2018_ratio.sort_values(by='area', inplace=True)

    #group by and merge
    nba_area_grp =  nba_df_2018_ratio.groupby("area", as_index=False).agg({"ratio" : np.average})
    nba_df = pd.merge(cities_nba, nba_area_grp, how = 'left', on = 'area')
    nba_df = nba_df.astype({'pop': int, 'ratio': float})
    
#     raise NotImplementedError()
    
    population_by_region = list(nba_df['pop']) # pass in metropolitan area population from cities
    win_loss_by_region =  list(nba_df['ratio']) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    #1. cleaning mlb data
    mlb_df_2018 = mlb_df[mlb_df['year']==2018]
    mlb_df_2018["ratio"] = pd.to_numeric(mlb_df_2018["W"]) / ( pd.to_numeric(mlb_df_2018["W"])+ pd.to_numeric(mlb_df_2018["L"]))
    mlb_df_2018 = mlb_df_2018[['team','ratio']]
    mlb_df_2018.replace('\*','', regex=True, inplace=True)
    mlb_df_2018.replace('\s\(\d+\)','', regex=True, inplace=True)
    mlb_df_2018_ratio = mlb_df_2018[['team','ratio']]
    mlb_df_2018_ratio.sort_values(by='team')

    #2. cities data
    cities.rename(columns={'Metropolitan area':'area' ,'Population (2016 est.)[8]':'pop'}, inplace=True)
    cities_mlb = cities[['area','pop','MLB']]
    cities_mlb.replace('\[note \d*\]', '', regex=True, inplace=True)
    cities_mlb.replace('', np.NaN, inplace=True)
    cities_mlb.replace('—', np.NaN, inplace=True)
    cities_mlb = cities_mlb[~cities_mlb['MLB'].isna()]
    cities_mlb.sort_values(by='area', inplace = True)

    #3. changing area name in nhl data
    one_name_area = ['Boston','Chicago', 'Toronto'] 
    mlb_df_2018_ratio['area'] =  mlb_df_2018_ratio['team'].apply(lambda x : re.split(' ',x)[0] if (len(re.split(' ',x)) == 2 or re.split(' ',x)[0] in one_name_area) else ' '.join(re.split(' ',x)[:2]))
    change_val = {'Arizona' : 'Phoenix'
                 , 'Colorado' : 'Denver'
                 , 'Miami' : 'Miami–Fort Lauderdale'
                 , 'Minnesota' : 'Minneapolis–Saint Paul'
                 , 'New York' : 'New York City'
                 , 'Oakland' : 'San Francisco Bay Area'
                 , 'San Francisco' : 'San Francisco Bay Area'
                 , 'Tampa Bay' : 'Tampa Bay Area'
                 , 'Texas' : 'Dallas–Fort Worth'
                 , 'Washington' : 'Washington, D.C.'}
    mlb_df_2018_ratio.replace({'area' : change_val }, inplace=True)
    mlb_df_2018_ratio.sort_values(by='area', inplace=True)
    mlb_df_2018_ratio

    #group by and merge
    mlb_area_grp =  mlb_df_2018_ratio.groupby("area", as_index=False).agg({"ratio" : np.average})
    mlb_df = pd.merge(cities_mlb, mlb_area_grp, how = 'left', on = 'area')
    mlb_df = mlb_df.astype({'pop': int, 'ratio': float})


#     raise NotImplementedError()
    
    population_by_region = list(mlb_df['pop']) # pass in metropolitan area population from cities
    win_loss_by_region = list(mlb_df['ratio']) # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    #1. cleaning nfl data
    nfl_df_2018 = nfl_df[nfl_df['year']==2018]
    remove_index = [0,5,10,15,20,25,30,35] #to remove rows with characters
    nfl_df_2018 = nfl_df_2018[~nfl_df_2018.index.isin(remove_index)]
    nfl_df_2018["ratio"] = pd.to_numeric(nfl_df_2018["W"]) / ( pd.to_numeric(nfl_df_2018["W"])+ pd.to_numeric(nfl_df_2018["L"]))
    nfl_df_2018 = nfl_df_2018[['team','ratio']]
    nfl_df_2018.replace('\*','', regex=True, inplace=True)
    nfl_df_2018.replace('\s\(\d+\)','', regex=True, inplace=True)
    nfl_df_2018_ratio = nfl_df_2018[['team','ratio']]
    nfl_df_2018_ratio.sort_values(by='team', inplace=True)

    #2. cities data
    cities.rename(columns={'Metropolitan area':'area' ,'Population (2016 est.)[8]':'pop'}, inplace=True)
    cities_nfl = cities[['area','pop','NFL']]
    cities_nfl.replace('\[note \d*\]', '', regex=True, inplace=True)
    cities_nfl.replace('', np.NaN, inplace=True)
    cities_nfl.replace('—', np.NaN, inplace=True)
    cities_nfl.replace('— ', np.NaN, inplace=True)
    cities_nfl = cities_nfl[~cities_nfl['NFL'].isna()]
    cities_nfl.sort_values(by='area', inplace = True)

    #3. changing area name in nhl data
    one_name_area = [''] 
    nfl_df_2018_ratio['area'] =  nfl_df_2018_ratio['team'].apply(lambda x : re.split(' ',x)[0] if (len(re.split(' ',x)) == 2 or re.split(' ',x)[0] in one_name_area) else ' '.join(re.split(' ',x)[:2]))
    change_val = {'Arizona' : 'Phoenix'
     , 'Carolina' : 'Charlotte'
     , 'Dallas' : 'Dallas–Fort Worth' 
     , 'Miami' : 'Miami–Fort Lauderdale'
     , 'Minnesota' : 'Minneapolis–Saint Paul'
     , 'New England' : 'Boston'
     , 'New York' : 'New York City'
     , 'Oakland' : 'San Francisco Bay Area'
     , 'San Francisco' : 'San Francisco Bay Area'
     , 'Tampa Bay' : 'Tampa Bay Area'
     , 'Tennessee' : 'Nashville'
     , 'Washington' : 'Washington, D.C.'}
    nfl_df_2018_ratio.replace({'area' : change_val }, inplace=True)
    nfl_df_2018_ratio.sort_values(by='area', inplace=True)

    #group by and merge
    nfl_area_grp =  nfl_df_2018_ratio.groupby("area", as_index=False).agg({"ratio" : np.average})
    nfl_df = pd.merge(cities_nfl, nfl_area_grp, how = 'left', on = 'area')
    nfl_df = nfl_df.astype({'pop': int, 'ratio': float})

#     raise NotImplementedError()
    
    population_by_region = list(nfl_df['pop']) # pass in metropolitan area population from cities
    win_loss_by_region = list(nfl_df['ratio']) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values